In [2]:
# Importer les packages
import json
import string
import random
import nltk
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to C:\Users\Imam
[nltk_data]     said\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Imam
[nltk_data]     said\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
# Extraire le fichier intents.json
with open('intents.json') as f:
  data = json.load(f)

In [14]:
# Afficher data
data

{'intents': [{'tag': 'salutation',
   'patterns': ['Salut',
    'Salut',
    'Comment vas-tu',
    'Comment allez-vous',
    "Est-ce que quelqu'un est là?",
    'Hé',
    'Salut',
    'Bonjour',
    'Bonne journée'],
   'responses': ['Bonjour',
    'Salut',
    'Content de te voir',
    "C'est bon de te revoir",
    'Bonjour, comment puis-je vous aider ?']},
  {'tag': 'au revoir',
   'patterns': ['Au revoir',
    'À plus tard',
    'Au revoir',
    'Aller se faire cuire un œuf',
    "Jusqu'à la prochaine fois",
    'au revoir'],
   'responses': ['À bientôt!', 'Passe une bonne journée', 'Au revoir! ']},
  {'tag': 'merci',
   'patterns': ['Merci',
    'Merci',
    "C'est utile",
    'Génial merci',
    "Merci de m'avoir aidé"],
   'responses': ['Heureux de pouvoir vous aider!',
    'À tout moment!',
    'Avec plaisir!',
    'Je vous en prie!']},
  {'tag': 'pas de réponse',
   'patterns': [],
   'responses': ['Désolé, je ne peux pas te comprendre',
    "S'il vous plaît donnez-moi plus d'i

In [15]:
# Initialiser le lemmatizer
lem = WordNetLemmatizer()

classes = [] # liste des différents labels
words = []   # liste des différents mots
documentX = [] # liste des différents patterns
documentY = [] # liste des labels de chaque patterns
# Tokeniser les patterns et rajouter les questions dans documentX et les étiquettes dans documentY
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tkns = nltk.word_tokenize(pattern) # tokeniser les patterns
        words.extend(tkns)# étendre words
        documentX.append(pattern)# rajouter pattern dans documentX
        documentY.append(intent['tag'])# rajouter le label dans document


    if intent["tag"] not in classes:
       classes.append(intent['tag'])# rajouter les labels dans Classes



words =[lem.lemmatize(word.lower()) for word in words if word is not string.punctuation] # Lemmatiser Words tout en s'assurant que les mots sont lowercase et les ponctuations ignorées
words =sorted(set(words))# récupérer les mots uniques et ordonner en ordre alphabétiques
classes = sorted(set(classes))# récupérer les labels uniques et  ordonner en ordre alphabétiques

In [16]:
train_data = [] # Liste des données train
output_vec = [0] * len(classes) # vecteur de sortie
# vectoriser les phrases de nos patterns
vectorizer = CountVectorizer(vocabulary= words)# Initialiser le vectorizer avec vocabulary=words
sentences_object = vectorizer.fit_transform(documentX) # Vectoriser documentX
sentences_array = sentences_object.toarray() # convertir en array
for idx, doc in enumerate(documentX):
    output_row = list(output_vec)# convertir output_vec en liste et l'affecter à output_row
    output_row[classes.index(documentY[idx])] = 1# créer le vecteur label pour la phrase d'index idx
    train_data.append([sentences_array[idx], output_row])# rajouter la liste contenant le vecteur phrase et son label sur train_data



random.shuffle(train_data)# Faire un shuffling de train_data
train_data = np.array(list(train_data))# convertir train_data en array

# Fractionner les données train_data
x =np.array(list(train_data[:, 0]))
y =np.array(list(train_data[:, 1]))

<ipython-input-16-de8d95870274>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_data = np.array(list(train_data))# convertir train_data en array


In [17]:
# Définir certains paramètres
input_shape = (len(x[0]),)
output_shape = len(y[0])
# Architecture de notre modèle
model = Sequential()

# Première couche à 128 neurones
model.add(Dense(128, input_shape = input_shape, activation = 'relu'))
# Couche Dropout avec comme learning_rate = 0.5
model.add(Dropout(0.5))
# Deuxième couche à 64 neurones
model.add(Dense(64, activation = 'relu'))
# Couche Dropout avec comme learning_rate = 0.3
model.add(Dropout(0.3))
# Couche de sortie
model.add(Dense(output_shape, activation = 'softmax'))
# Compiler le modèle
model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics =['accuracy'])
# Afficher le summary du modèle
model.summary()
# Entrainer le modèle avec 200 epochs
model.fit(x,y ,epochs = 200)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               20736     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 30)                1950      
                                                                 
Total params: 30942 (120.87 KB)
Trainable params: 30942 (120.87 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
4/4 

In [18]:
model.evaluate(x,y)

4/4 [==============================] - 0s 4ms/step - loss: 0.0713 - accuracy: 0.9569


[0.07133887708187103, 0.9568965435028076]

In [19]:
# Fonction tokenisation et lemmatisant le pattern entrée par le user
def tk_lm_func(text):
  tkns = nltk.word_tokenize(text)
  tkns = [lem.lemmatize(word) for word in tkns]
  return tkns

# fonction vectorisant la question entrée par le user
def vectorizer_func(text, vocab):
  tkns = tk_lm_func(text)
  sent_vec = [0] * len(vocab)
  for w in tkns:
    for idx, word in enumerate(vocab):
      if word == w:
        sent_vec[idx] = 1
  return np.array(sent_vec)

# Fonction de prédiction
def Pred_func(text, vocab, labels):
  sent_vec = vectorizer_func(text, vocab)# Vectoriser la phrase
  result =model.predict(np.array([sent_vec]))  # Prédire
  result = result.argmax(axis =1) # Récupérer l'index du label qui la probabilité la plus grande
  tag =labels[result[0]]  # Récupérer le label en text
  return tag# Retourner tag

def get_res(tag, fJson):
  list_intents = fJson['intents'] # liste des intents
  for i in list_intents:
    if i["tag"] == tag:
      ourResult = random.choice(i['responses']) # Choisir de aléatoire par les responses de label tag
      break
  return ourResult

In [ ]:
# Lancer le chatting
while True:
  Newmessage = input('')
  if Newmessage.lower() == 'quit':
    break
  print('User : ', Newmessage)
  tag = Pred_func( Newmessage, words, classes)
  result = get_res(tag, data)
  print('Chatbot :', result)

Est-ce que quelqu'un est là?
User :  Est-ce que quelqu'un est là?
1/1 [==============================] - 0s 19ms/step
Chatbot : Salut
Qui es-tu
User :  Qui es-tu
1/1 [==============================] - 0s 22ms/step
Chatbot : Veuillez envoyer vos suggestions à wahab@gmail.com. 
Ce que tu peux faire?
User :  Ce que tu peux faire?
1/1 [==============================] - 0s 20ms/step
Chatbot : Je suis un chatbot à usage général. 
Qui t'a fait
User :  Qui t'a fait
1/1 [==============================] - 0s 20ms/step
Chatbot : J'ai été réalisé par Abdoul Wahab.
que fais-tu
User :  que fais-tu
1/1 [==============================] - 0s 37ms/step
Chatbot : Je suis Wahab, un chatbot Deep Learning
Super
User :  Super
1/1 [==============================] - 0s 19ms/step
Chatbot : Ouais!
Raconte moi une blague
User :  Raconte moi une blague
1/1 [==============================] - 0s 20ms/step
Chatbot : Que seraient les ours sans les abeilles ? 
haha
User :  haha
1/1 [==============================] - 0s